# Chapter 9 딥러닝의 미래 GAN

GAN(Generative Adversarial Network)는 서로 대립하는 두 신경망을 경쟁시켜가며 결과물을 생성하는 방법을 학습하는 모델입니다.

> 위조지폐범(생성자)와 경찰(구분자)가 있다고 가정합시다. 위조지폐범은 경찰을 속이려고 노력하고, 경찰은 위조한 지폐를 감별하려고 최대한 노력합니다. 이를 통해 서로의 능력이 발전하고, 위조지폐범은 진짜와 거의 구별되지 않는 위조지폐를 만들 수 있게 됩니다.

신경망 모델로 나타내면 아래와 같습니다

![GAN_1](GAN_1.png)

1. 먼저 실제 이미지를 주고 구분자(Discriminator)에게 이 이미지가 진짜임을 판단하게 합니다.
2. 생성자(Generator)를 통해 임의의 이미지를 만들고 이것을 다시 같은 구분자를 통해 진짜 이미지인지를 판단하게 합니다.
3. 생성자는 구분자를 속여 진짜처럼 보이게 하고, 구분자는 생성자가 만든 이미지를 최대한 가짜라고 구분하도록 훈련한다.

이런 경쟁을 통해 생성자는 실제 이미지와 상당히 비슷한 이미지를 생성하게 된다.

1. 사진을 고흐 풍 그림으로 다시 그리기
2. 선으로만 그려진 만화를 채색
3. 모자이크 없애기
4. 자연어 문장 생성 등등

![GAN_2](GAN_2.png)

이번 장에서는 GAN 모델을 이용해 MNIST 손글씨를 무작위로 생성하고, 이를 학습하여 원하는 숫자의 이미지를 생성하는 모델을 만들어보겠습니다.

## 9.1 GAN 기본 모델 구현하기



In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 255
n_input = 28 * 28
n_noise = 128 # 생성자의 입력값으로 사용할 노이즈의 크기

In [3]:
# GAN도 비지도 학습이므로 오토인코더처럼 Y(Labeling) 데이터를 사용하지 않습니다.

# 이미지를 넣을 X와 이미지에 들어가는 노이즈를 넣을 Z를 추가한다.
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [4]:
# 생성자 신경망에 사용할 변수 설정
# 노이즈로 이미지를 생성
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [5]:
# 구분자 신경망에 사용할 변수 설정
# 이미지를 넣고 가짜인지 진짜인지 판별
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

구분자는 실제 이미지와 생성한 이미지를 받아 가짜인지를 구별한다

In [6]:
# 생성자 신경망
# 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 
# 실제 이미지와 같은 크기의 가짜 결괏값을 반환한다.
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

In [ ]:
# 구분자 신경망
# 0~1 사이의 